In [ ]:
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"



# Spark ML Selección de Variables

Cargamos un dataset con información sobre partos. Este dataset tiene como variable objetivo el fallecimiento o supervivencia de los recién nacidos.



### Crear SparkSession

In [ ]:
# Respuesta

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



### Cargar datos y comprobar schema

In [ ]:
# Respuesta

births = spark.read.csv(DATA_PATH+'data/births_train.csv',sep=',', header=True, inferSchema=True) # You may try with: births.csv

births.printSchema()

In [ ]:
# Respuesta

births.show(1)

 

Se puede ver la variable objetivo (_target_) "INFANT_ALIVE_AT_REPORT" es de tipo "string".  Para realizar los siguientes análisis la misma debe ser convertida a tipo numérico.

In [ ]:
# Respuesta

births.select('INFANT_ALIVE_AT_REPORT').distinct().show()

In [ ]:
# Respuesta

from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

births= births.withColumn("INFANT_ALIVE_AT_REPORT", F.udf(lambda x: 1.0 if x == "Y" else 0.0, FloatType())("INFANT_ALIVE_AT_REPORT") )

births.groupby("INFANT_ALIVE_AT_REPORT").count().show()



Todas las variables de tipo 'string' son categóricas. Hagamos un StringIndexer con ellas

In [ ]:
# Respuesta

from pyspark.ml.feature import StringIndexer

stringindexer_dictionary ={}




Es una buena idea guardar todos los modelos para poder acceder a los cambios para cada categoría

In [ ]:
# Respuesta

for element in births.dtypes:
    if element[1] == 'string':
        stringindexer = StringIndexer(inputCol = element[0], outputCol=element[0]+'_indexed')
        stringindexer_model = stringindexer.fit(births)
        births = stringindexer_model.transform(births)
        births = births.drop(element[0])
        
        stringindexer_dictionary[element[0]] = stringindexer_model

In [ ]:
# Respuesta

stringindexer_dictionary



### ChiSquared

Este método de selección de variables se aplica a variables categóricas.

In [ ]:
# Respuesta

from pyspark.ml.feature import ChiSqSelector, VectorAssembler



El primer paso es crear un VectorAssembler de las variables categóricas

In [ ]:
# Respuesta

input_cols = [element for element in births.columns if '_indexed' in element]

vectorassembler = VectorAssembler(inputCols=input_cols, outputCol='categorical_assembled')
births = vectorassembler.transform(births)



Chi Squared selector

In [ ]:
# Respuesta

chisquared = ChiSqSelector(featuresCol=vectorassembler.getOutputCol(), labelCol="INFANT_ALIVE_AT_REPORT", numTopFeatures=5)
chisquared_model = chisquared.fit(births)



Ver resultado:

In [ ]:
# Respuesta

chisquared_model.selectedFeatures



Ver nombre de las variables:



Nota: el orden viene dado por el orden en que se han introducido las variables en el VectorAssembler.

In [ ]:
# Respuesta

top_5_categorical = [input_cols[index] for index in chisquared_model.selectedFeatures]
top_5_categorical



### RandomForest

Este método toma tanto variables categóricas como numéricas. Tiene el incoveniente que es aleatorio (_random_) y los resultados pueden verse modificados. Vamos a ver como contrarrestar este problema. Además, retorna importancia de las variables.



Importamos RandomForestClassifier puesto que se trata de un problema de clasificación

In [ ]:
# Respuesta

from pyspark.ml.classification import RandomForestClassifier



Para empezar, seleccionamos todas las variables menos la objetivo y la columna vector. Luego debemos volver a crear un VectorAssembler y ejecutamos el algoritmo RandomForestClassifier.

In [ ]:
# Respuesta

features_for_rf = [element for element in births.columns if element != 'INFANT_ALIVE_AT_REPORT' and element !='categorical_assembled']

In [ ]:
# Respuesta

vectorassembler = VectorAssembler(inputCols=features_for_rf, outputCol='assembled_rf')
births = vectorassembler.transform(births)

rf = RandomForestClassifier(featuresCol=vectorassembler.getOutputCol(), labelCol='INFANT_ALIVE_AT_REPORT')

rf_model = rf.fit(births)



De nuevo, el índice del vector viene dado por el orden de entrada de las variables en el VectorAssembler

In [ ]:
# Respuesta

rf_model.featureImportances



* Ordenamos importancias para quedarnos con las variables que explican el x% de importancia respecto la variable objetivo. Tomaremos el 95% de importancia



Primero, creamos una lista y almacenamos el índice para poder recuperar el nombre de la variable más adelante


In [ ]:
# Respuesta

importances = [(index, value) for index, value in enumerate(rf_model.featureImportances.toArray().tolist())]



Ordenamos de mayor a menor importancia

In [ ]:
# Respuesta

importances = sorted(importances, key=lambda value: value[1], reverse=True)



Nos quedamos con aquellas que explican el 95% de las variables

In [ ]:
# Respuesta

compt = 0
important_features =[]
for element in importances:
    if compt < 0.95:
        compt += element[1]
        important_features.append((features_for_rf[element[0]], element[1]))

In [ ]:
# Respuesta

important_features



Ahora, veamos como evitar la aleatoriedad del RandomForest haciendo varias iteraciones y quedándonos con las variables que aparecen en todas las iteraciones



Creemos una semilla (_seed_), al inicializarlo será replicable.

In [ ]:
# Respuesta

random_seed = 4
num_iter = 10

import random

In [ ]:
# Respuesta

def feature_importance_RF(random_seed, featuresCol, featuresCols, labelCol, df):
    rf = RandomForestClassifier(featuresCol=featuresCol, labelCol=labelCol, seed = random_seed )
    rf_model = rf.fit(df)
    
    importances = [(index, value) for index, value in enumerate(rf_model.featureImportances.toArray().tolist())]

    #order from highest to lowest importance
    importances = sorted(importances, key=lambda value: value[1], reverse=True)

    #select those that explain 95% of the target variable

    compt = 0
    important_features =[]
    for element in importances:
        if compt < 0.95:
            compt += element[1]
            important_features.append((featuresCols[element[0]], element[1]))
    return important_features

In [ ]:
# Respuesta

random.seed(random_seed)

random_seeds=[]

while len(set(random_seeds)) < num_iter:
    random_seeds.append(random.randint(0,10000))

features_random_seed = []
for random_seed in random_seeds:
    features_random_seed.append(feature_importance_RF(random_seed, vectorassembler.getOutputCol(), features_for_rf,'INFANT_ALIVE_AT_REPORT', births))

In [ ]:
# Respuesta

features_random_seed



Tras construir esta lista de listas con información de cada iteración, nos quedamos con aquellas variables que aparecen en cada iteración. Veamos cómo se hace.



Primero convertimos la lista de listas en una sola lista

In [ ]:
# Respuesta

flat_features = [feature for one_seed in features_random_seed for feature in one_seed]
features = [element[0] for element in flat_features]

from collections import Counter

features_all_seeds = [element[0] for element in Counter(features).items() if element[1] == num_iter]
features_all_seeds



Si quisiéramos el valor de la importancia en sí, calculamos media por cada semilla:

In [ ]:
# Respuesta

import numpy as np

dictionary_importances = {}

for feature in features_all_seeds:
    dictionary_importances[feature] = []
    for values in features_random_seed:
        for element in values:
            if element[0] == feature:
                dictionary_importances[feature].append(element[1])
                break
    dictionary_importances[feature] = np.mean(dictionary_importances[feature])

In [ ]:
# Respuesta

sorted(dictionary_importances.items(), key=lambda value: value[1], reverse=True)